##### Generate the EMA indicater with talib package. Generate signal with my logic

In [1]:
from dotenv import find_dotenv, load_dotenv
import os
import psycopg2
import pandas as pd
import numpy as np
from numba import njit
import talib
import pybroker 
from pybroker import Strategy, IndicatorSet, highest
from pybroker.data import DataSource, YFinance
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print(f"pandas version: {pd.__version__}")
print(f"pybroker version: {pybroker.__version__}")

pandas version: 2.2.3
pybroker version: 1.2.5


In [2]:
def load_data_from_postgres(db_host, db_name, db_user, db_password, table_name, symbol, start_date, end_date):
    """
    Loads data from a PostgreSQL database and returns a pybroker Data object.

    Args:
        db_host (str): Hostname of the PostgreSQL database.
        db_name (str): Name of the database.
        db_user (str): Username to connect to the database.
        db_password (str): Password to connect to the database.
        table_name (str): Name of the table containing the data.
        symbol (str): The symbol for the data.
        start_date (str): Start date for the data (YYYY-MM-DD).
        end_date (str): End date for the data (YYYY-MM-DD).

    Returns:
        pybroker.data.Data: A pybroker Data object.
    """
    conn = None  # Initialize conn to None
    try:
        conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
        curr = conn.cursor()
    # Construct the SQL query to fetch data within the specified date range
        query = f""" 
            SELECT ticker, date, open, high, low, closes, volume
            FROM {table_name}
            WHERE date >= '{start_date}' AND date <= '{end_date}' AND ticker ilike '{symbol}'
            ORDER BY date ASC;
        """
        curr.execute(query)
        data_row = curr.fetchall()

        if not data_row:
            print(f"No data found in table {table_name} for the given date range.")

        # Convert fetched data into pandas DataFrame
        # df = pd.DataFrame(data_row, columns=['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df = pd.DataFrame(data_row, columns=['symbol', 'date', 'open', 'high', 'low', 'close', 'volume'])


        # convert to float data type for these columns
        df['open'] = pd.to_numeric(df['open'], errors='coerce')
        df['high'] = pd.to_numeric(df['high'], errors='coerce')
        df['low'] = pd.to_numeric(df['low'], errors='coerce')
        df['close'] = pd.to_numeric(df['close'], errors='coerce')
        df['volume'] = pd.to_numeric(df['volume'], errors='coerce')

        # Set 'Date' as index and convert to datetime
        df['date'] = pd.to_datetime(df['date'])
        # df.set_index('date', inplace=True)

        # Create a pybroker Data object
        # pybroker_data = data.Data(symbol, df)
        return df
    
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL: {e}")
        return None # Return None if there was an error
    
    finally:
        if conn:
            curr.close()
            conn.close()




In [ ]:
## connect to 'blk' postgress database
db_host = os.getenv("db_host")
db_name = os.getenv("db_name")
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
    
table_name = 'vw_siamchart_par_adj'
symbol = 'PTTEP'
start_date = "2023-01-01"
end_date = "2024-12-31"

print(f"db_host: {db_host}\ndb_name: {db_name}\ndb_user: {db_user}\ndb_password: {db_password}")


db_host: localhost
db_name: blk
db_user: postgres
db_password: blk*4848


In [4]:
df = load_data_from_postgres(db_host, db_name, db_user, db_password, 
                             table_name, symbol, start_date, end_date)

df.head()

,symbol,date,open,high,low,close,volume
0,PTTEP,2023-01-03,178.0,178.5,175.0,175.0,6298100.0
1,PTTEP,2023-01-04,173.0,173.0,167.0,167.0,18276400.0
2,PTTEP,2023-01-05,167.0,170.0,165.5,168.5,17177000.0
3,PTTEP,2023-01-06,168.0,169.5,167.5,167.5,6477700.0
4,PTTEP,2023-01-09,169.0,169.5,167.0,168.5,7524500.0
